### Import Required Libraries and Set Up Environment Variables

In [61]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from pathlib import Path
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv("./.env")
# Takes 24 hours before key works
NASA_API_KEY = os.getenv('NASA_API_KEY')

### CME Data

In [62]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
# Coronal Mass Ejection:
# https://api.nasa.gov/DONKI/CME?startDate=yyyy-MM-dd&endDate=yyyy-MM-dd&api_key=DEMO_KEY
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
cme_url = f"{base_url}CME?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"
# print(cme_url)

In [33]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(cme_url)
cme_response


<Response [200]>

In [34]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()

In [35]:
# Preview the first result in JSON format
# cme_json[0]
# Use json.dumps with argument indent=4 to format data
print(json.dumps([cme_json[0]], indent=4))


[
    {
        "activityID": "2013-05-01T03:12:00-CME-001",
        "catalog": "M2M_CATALOG",
        "startTime": "2013-05-01T03:12Z",
        "instruments": [
            {
                "displayName": "SOHO: LASCO/C2"
            },
            {
                "displayName": "SOHO: LASCO/C3"
            },
            {
                "displayName": "STEREO A: SECCHI/COR2"
            },
            {
                "displayName": "STEREO B: SECCHI/COR2"
            }
        ],
        "sourceLocation": "",
        "activeRegionNum": null,
        "note": "",
        "submissionTime": "2013-08-07T16:54Z",
        "versionId": 1,
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
        "cmeAnalyses": [
            {
                "isMostAccurate": true,
                "time21_5": "2013-05-01T07:07Z",
                "latitude": 12.0,
                "longitude": -120.0,
                "halfAngle": 36.0,
                "speed": 860.0,
        

In [ ]:
# [Optional: Create DataFrame]
cme_json_df = pd.DataFrame(cme_json)
# Write an external file, so we don't have to wait for the request again:
# cme_json_df.to_json("./cme_data.json")


In [37]:
# Convert cme_json to a Pandas DataFrame 
# cme_df = pd.DataFrame(cme_json)

# [Optional: Read the Data from external file]
cme_df = pd.read_json("./cme_data.json", orient="records")
cme_df

# Keep only the columns: activityID, startTime, linkedEvents
cme_subset_df = cme_df[["activityID", "startTime", "linkedEvents"]].reset_index(drop=True)
cme_subset_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,2013-05-02T05:24Z,None
2,2013-05-02T14:36:00-CME-001,2013-05-02T14:36Z,None
3,2013-05-03T18:00:00-CME-001,2013-05-03T18:00Z,None
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]


In [38]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_clean_df = cme_subset_df.dropna(subset=['linkedEvents'])
cme_clean_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...


In [39]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index to traverse DataFrame records
for row in cme_clean_df.index:
    # Iterate over each dictionary in the list
    # print("row:", row)
    activity_id = cme_clean_df.loc[row,"activityID"]
    start_time = cme_clean_df.loc[row, "startTime"]
    linked_events = cme_clean_df.loc[row, "linkedEvents"]
    
    # Loop through linked_event
    for linked_event in linked_events:
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
        expanded_rows.append({"activityID" : activity_id,
                              "startTime": start_time,
                              "linkedEvents" : linked_event})
      
# Create a new DataFrame from the expanded rows
expanded_rows_df = pd.DataFrame(expanded_rows)
expanded_rows_df.head()
# expanded_rows_df.info()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'}
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'}
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'}
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'}
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'}


In [40]:
# Verify activityID column exists:
expanded_rows_df.get("activityID")

0       2013-05-01T03:12:00-CME-001
1       2013-05-03T22:36:00-CME-001
2       2013-05-09T19:29:00-CME-001
3       2013-05-13T02:54:00-CME-001
4       2013-05-13T02:54:00-CME-001
                   ...             
1709    2024-04-24T01:48:00-CME-001
1710    2024-04-24T15:05:00-CME-001
1711    2024-04-25T18:24:00-CME-001
1712    2024-04-30T00:36:00-CME-001
1713    2024-05-01T23:16:00-CME-001
Name: activityID, Length: 1714, dtype: object

In [41]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
def extract_activityID_from_dict(input_dict):
        try:
                return f'{input_dict.get("activityID")}'
        except KeyError: 
                # Log the error or print it for debugging
                print(f'Activity Id does not exist on this record')


# extract_activityID_from_dict(expanded_rows_df[0])
linked_event = expanded_rows_df.loc[0, "linkedEvents"]
# print("LinkedEvent:", linked_event)
activityId = extract_activityID_from_dict(linked_event)
activityId

'2013-05-04T04:52:00-IPS-001'

In [42]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
expanded_rows_df['GST_ActivityID'] = expanded_rows_df["linkedEvents"].apply(lambda x: extract_activityID_from_dict(x))
expanded_rows_df.head(5)

,activityID,startTime,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001


In [43]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
# This was already done in a previous step
pre_drop_count = expanded_rows_df["GST_ActivityID"].count()
clean_activityID_df = expanded_rows_df.dropna(subset = ['GST_ActivityID'])
post_drop_count = clean_activityID_df.count()
print("Pre_Drop_Count", pre_drop_count)
print("Post_Count:", post_drop_count)

Pre_Drop_Count 1714
Post_Count: activityID        1714
startTime         1714
linkedEvents      1714
GST_ActivityID    1714
dtype: int64


In [44]:
# print out the datatype of each column in this DataFrame:
clean_activityID_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   activityID      1714 non-null   object
 1   startTime       1714 non-null   object
 2   linkedEvents    1714 non-null   object
 3   GST_ActivityID  1714 non-null   object
dtypes: object(4)
memory usage: 53.7+ KB


In [45]:
#### Note: RangeIndex will not match original: 1 of 4 #######
# Convert the 'GST_ActivityID' column to string format 
clean_activityID_df['GST_ActivityID'] = clean_activityID_df['GST_ActivityID'].astype('string')
# print(cleaned_activityID_df)

# Convert startTime to datetime format and maintain timezone
clean_activityID_df['startTime'] = pd.to_datetime(clean_activityID_df['startTime'], utc=True)

# Rename startTime to startTime_CME and activityID to cmeID
renamed_activityID_df = clean_activityID_df.rename(columns={ "startTime": "startTime_CME",
                                                              "activityID" : "cmeID"})
# Drop linkedEvents
del renamed_activityID_df['linkedEvents']

# Verify that all steps were executed correctly
renamed_activityID_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   cmeID           1714 non-null   object             
 1   startTime_CME   1714 non-null   datetime64[ns, UTC]
 2   GST_ActivityID  1714 non-null   string             
dtypes: datetime64[ns, UTC](1), object(1), string(1)
memory usage: 40.3+ KB


In [46]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
renamed_activityID_df = renamed_activityID_df[renamed_activityID_df["GST_ActivityID"].str.contains('GST', na=False)]
renamed_activityID_df.head()

,cmeID,startTime_CME,GST_ActivityID
21,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
48,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
90,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
148,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
151,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001


### GST Data

In [47]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
gst_url = f"https://api.nasa.gov/DONKI/{GST}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"
# print(gst_url)


In [48]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(gst_url)
gst_response

<Response [200]>

In [49]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()
# Preview the first result in JSON format
first_result = gst_json[0]
# print(first_result)
# Use json.dumps with argument indent=4 to format data
print(json.dumps([first_result], indent=4))

[
    {
        "gstID": "2013-06-01T01:00:00-GST-001",
        "startTime": "2013-06-01T01:00Z",
        "allKpIndex": [
            {
                "observedTime": "2013-06-01T01:00Z",
                "kpIndex": 6.0,
                "source": "NOAA"
            }
        ],
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
        "linkedEvents": [
            {
                "activityID": "2013-05-31T15:45:00-HSS-001"
            }
        ],
        "submissionTime": "2013-07-15T19:26Z",
        "versionId": 1
    }
]


In [ ]:
# Convert gst_json to a Pandas DataFrame  
gst_json_df = pd.DataFrame(gst_json)
# Write an external file, so we don't have to wait for the request again:
# gst_json_df.to_json("./gst_data.json")
# Read the Data from external file
gst_df = pd.read_json("./gst_data.json", orient="records")
gst_df
# Keep only the columns: gstID, startTime, linkedEvents
gst_subset_df = gst_df[["gstID", "startTime", "linkedEvents"]].reset_index(drop=True)
gst_subset_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-06-29T03:00:00-GST-001,2013-06-29T03:00Z,None
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...


In [51]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_clean_df = gst_subset_df.dropna(subset=['linkedEvents'])
gst_clean_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
5,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...


In [52]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_exploded_df = gst_clean_df.explode("linkedEvents", ignore_index=True).dropna()
# Use the had function to show the dataframe
gst_exploded_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'}
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'}
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'}
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'}
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'}


In [53]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
gst_exploded_df['CME_ActivityID'] = gst_exploded_df["linkedEvents"].apply(lambda x: extract_activityID_from_dict(x))
gst_exploded_df.head(5)
# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
clean_activityID_df = gst_exploded_df.dropna(subset = ['CME_ActivityID'])
clean_activityID_df.head()


,gstID,startTime,linkedEvents,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'},2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'},2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'},2013-10-02T02:47:00-MPC-001


In [54]:
#### Note: RangIndex will not match original 2 of 4 #####
# Convert the 'CME_ActivityID' column to string format 
clean_activityID_df['CME_ActivityID'] = clean_activityID_df['CME_ActivityID'].astype('string')
# Convert the 'gstID' column to string format 
clean_activityID_df['gstID'] = clean_activityID_df['gstID'].astype('string')
# Convert startTime to datetime format  
clean_activityID_df['startTime'] = pd.to_datetime(clean_activityID_df['startTime'], utc=True)

# Rename startTime to startTime_GST 
renamed_start_time_df = clean_activityID_df.rename(columns={ "startTime": "startTime_GST"})
# Drop linkedEvents
del renamed_start_time_df['linkedEvents']
# Verify that all steps were executed correctly
renamed_start_time_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           205 non-null    string             
 1   startTime_GST   205 non-null    datetime64[ns, UTC]
 2   CME_ActivityID  205 non-null    string             
dtypes: datetime64[ns, UTC](1), string(2)
memory usage: 4.9 KB


In [55]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
renamed_start_time_df = renamed_start_time_df[renamed_start_time_df["CME_ActivityID"].str.contains('CME', na=False)]
renamed_start_time_df.head()


,gstID,startTime_GST,CME_ActivityID
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
5,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
7,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
9,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001


### Merge both datatsets

In [56]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_cme_gst_df = pd.merge(renamed_start_time_df, renamed_activityID_df, how='inner', left_on=['gstID', 'CME_ActivityID'], right_on=['GST_ActivityID', 'cmeID'])
merged_cme_gst_df.head()

,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
2,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
3,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
4,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001


In [57]:
##### Note: RangeIndex will not match the original: 3 of 4 ########
# Verify that the new DataFrame has the same number of rows as cme and gst
print("merged_df:", merged_cme_gst_df.info())
# print("gst:", renamed_start_time_df.info())
# print("cme:", renamed_activityID_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           61 non-null     string             
 1   startTime_GST   61 non-null     datetime64[ns, UTC]
 2   CME_ActivityID  61 non-null     object             
 3   cmeID           61 non-null     object             
 4   startTime_CME   61 non-null     datetime64[ns, UTC]
 5   GST_ActivityID  61 non-null     string             
dtypes: datetime64[ns, UTC](2), object(2), string(2)
memory usage: 3.0+ KB
merged_df: None


### Computing the time it takes for a CME to cause a GST

In [58]:
# Convert both columns to datetime
merged_cme_gst_df["startTime_CME"] = pd.to_datetime(merged_cme_gst_df["startTime_CME"], utc=True)
merged_cme_gst_df["startTime_GST"] = pd.to_datetime(merged_cme_gst_df["startTime_GST"], utc=True)

# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_cme_gst_df["Time Diff"] = merged_cme_gst_df["startTime_GST"] - merged_cme_gst_df["startTime_CME"]
merged_cme_gst_df.head()

,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID,Time Diff
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001,4 days 06:36:00
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001,2 days 04:20:00
2,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001,3 days 00:48:00
3,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001,2 days 12:45:00
4,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001,2 days 01:35:00


In [59]:
### Note: Count will not match the original: 4 of 4 ####
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
# merged_cme_gst_df.describe()
merged_cme_gst_df["Time Diff"].describe()

count                           61
mean     2 days 23:29:26.557377049
std      0 days 23:53:09.336914240
min                1 days 08:36:00
25%                2 days 04:00:00
50%                2 days 19:51:00
75%                3 days 13:35:00
max                6 days 03:00:00
Name: Time Diff, dtype: object

### Exporting data in csv format

In [60]:
# Export data to CSV without the index
file = Path("./Resources/cme_gst.csv")
merged_cme_gst_df.to_csv(file, index=False)